In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
from v4vapp_backend_v2.config.setup import InternalConfig
from v4vapp_backend_v2.database.db import get_mongodb_client_defaults
from v4vapp_backend_v2.actions.tracked_models import TrackedBaseModel

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())


Current working directory: /Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2


In [18]:
from v4vapp_backend_v2.accounting.balance_sheet import (
    generate_balance_sheet_pandas,
    get_account_balance_printout,
    get_ledger_dataframe,
    list_all_accounts,
)


CONFIG = InternalConfig(config_filename="devhive.config.yaml").config

db_client = get_mongodb_client_defaults()

TrackedBaseModel.db_client = db_client
ledger_df = await get_ledger_dataframe()
balance_sheet_dict = await generate_balance_sheet_pandas(df=ledger_df)


all_accounts = await list_all_accounts()
print("All accounts:", all_accounts)
balances = {}
for account in all_accounts:
    print(f"Account: {account}")
    balances[str(account)] = await get_account_balance_printout(
        account=account, full_history=True, df=ledger_df
    )
for account, balance in balances.items():
    print(f"Balance for {account}:")
    print(balance)

2025-06-10T18:22:31+0300.750 INFO     db                        161 : 📁 backend Initializing MongoDBClient local_connection
2025-06-10T18:22:31+0300.771 INFO     db                        557 : 📁 backend Attempting to connect to MongoDB for the first time or after failure.
2025-06-10T18:22:32+0300.729 INFO     balance_sheet             547 : Balance Sheet is balanced. Assets 124.16 USD


All accounts: [Customer Deposits Hive (Asset) - Sub: devser.v4vapp, Escrow Hive (Asset) - Sub: devser.v4vapp, Treasury Hive (Asset) - Sub: devtre.v4vapp, Treasury Lightning (Asset) - Sub: umbrel, Customer Liability Hive (Liability) - Sub: v4vapp-test, Owner Loan Payable (funding) (Liability) - Sub: umbrel, Owner Loan Payable (funding) (Liability) - Sub: v4vapp.dhf, Fee Income Lightning (Revenue) - Sub: umbrel]
Account: Customer Deposits Hive (Asset) - Sub: devser.v4vapp
Account: Escrow Hive (Asset) - Sub: devser.v4vapp
Account: Treasury Hive (Asset) - Sub: devtre.v4vapp
Account: Treasury Lightning (Asset) - Sub: umbrel
Account: Customer Liability Hive (Liability) - Sub: v4vapp-test
Account: Owner Loan Payable (funding) (Liability) - Sub: umbrel
Account: Owner Loan Payable (funding) (Liability) - Sub: v4vapp.dhf
Account: Fee Income Lightning (Revenue) - Sub: umbrel
Balance for Customer Deposits Hive (Asset) - Sub: devser.v4vapp:
Balance for Customer Deposits Hive (sub: devser.v4vapp) As

Balance for Customer Deposits Hive (Asset) - Sub: devser.v4vapp:
Balance for Customer Deposits Hive (sub: devser.v4vapp) Asset
-----------------------------------------------------------------------------------------------------------------------------

Unit: HIVE
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-06-09 11:40:54  Treasury to Server transfer: 50.000 HIVE f...        50.00         0.00        50.00  96629458_6ce61
2025-06-10 07:39:21  4.445  HBD -->     18.292 HIVE droida fill...        18.29         0.00        68.29  96653393_ae62c
2025-06-10 08:28:12  Deposit: 16.000 HIVE from v4vapp-test to d...        16.00         0.00        84.29  96654370_79508
2025-06-10 08:28:50  Conv 15.509 HIVE to 3,434 sats devser.v4vapp          0.00        15.51        68.78      1f8eaa549a
2025-06-10 08:29:00  Withdrawal: 0.491 HIVE from devser.v4vapp ...         0.00         0.49        68.29  96654386_232

In [5]:
from v4vapp_backend_v2.accounting.balance_sheet import (
    balance_sheet_all_currencies_printout,
    balance_sheet_printout,
)


balance_sheet_str = balance_sheet_all_currencies_printout(
    balance_sheet_dict,
)
print(balance_sheet_str)

Balance Sheet in All Currencies
-------------------------------------------------------------------------------------------------------------------
Account                                  Sub                     SATS         HIVE          HBD          USD
-------------------------------------------------------------------------------------------------------------------

                                                       Assets                                                      
------------------------------
Customer Deposits Hive                   devser.v4vapp         56,967       256.62        62.07        62.07
Total Customer Deposits Hive                                   56,967       256.62        62.07        62.07
Treasury Hive                            devtre.v4vapp         56,986       256.70        62.09        62.09
Total Treasury Hive                                            56,986       256.70        62.09        62.09
-------------------------------------------

In [6]:
from v4vapp_backend_v2.accounting.balance_sheet import get_ledger_entries


ledger_entries = await get_ledger_entries()
for ledger_entry in ledger_entries:
    print(ledger_entry)


J/E NUMBER: 96628404_dc6ffeb9f87f107ee7a0d3d8bf892d5068ca4ba0_1_real 96628404_dc6ff
DATE
Jun 09, 2025 10:48  

ACCOUNT                                                                 DEBIT          CREDIT
----------------------------------------------------------------------------------------------------
Treasury Hive (Asset)                           devtre.v4vapp     100.00 hive                
    Owner Loan Payable (funding) (Liability)           v4vapp.dhf                     100.00 hive

DESCRIPTION
💬Funding to Treasury transfer: 100.000 HIVE from v4vapp.dhf to devtre.v4vapp 💬Restart testing
balances for V2


J/E NUMBER: 96628490_e4c5be7ec6bed3819a984e417722dfa47a1c0852_1_real 96628490_e4c5b
DATE
Jun 09, 2025 10:52  

ACCOUNT                                                                 DEBIT          CREDIT
----------------------------------------------------------------------------------------------------
Treasury Hive (Asset)                           devtre.v4vapp      100

In [7]:
print(balances["Customer Liability Hive (Liability) - Sub: v4vapp-test"])


Balance for Customer Liability Hive (sub: v4vapp-test) Liability
-----------------------------------------------------------------------------------------------------------------------------

Unit: HBD
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-06-09 11:43:36  Deposit: 5.000 HBD from v4vapp-test to dev...         0.00         5.00         5.00  96629512_7601a
2025-06-09 11:44:03  Withdrawal: 5.000 HBD from devser.v4vapp t...         5.00         0.00         0.00  96629521_9ebb6
2025-06-10 07:20:39  Deposit: 5.000 HBD from v4vapp-test to dev...         0.00         5.00         5.00  96653019_0728e
2025-06-10 07:22:21  Withdrawal: 5.000 HBD from devser.v4vapp t...         5.00         0.00         0.00  96653053_b75be
2025-06-10 07:28:24  Deposit: 3.000 HBD from v4vapp-test to dev...         0.00         3.00         3.00  96653174_1157b
2025-06-10 07:31:18  Withdrawal: 3.000 HBD from devser.v4vap

In [8]:
print(balances["Fee Income Lightning (Revenue) - Sub: umbrel"])

Balance for Fee Income Lightning (sub: umbrel) Revenue
-----------------------------------------------------------------------------------------------------------------------------

Unit: SATS
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-06-10 07:38:30  Fee Lightning v4vapp-test 3,948 sats                     0          120          120      18642d3ac1
2025-06-10 08:28:50  Fee Lightning v4vapp-test 3,324 sats                     0          110          230      1f8eaa549a
-----------------------------------------------------------------------------------------------------------------------------
Converted                1.04 HIVE         0.25 HBD         0.25 USD          230 SATS          230,281 msats
-----------------------------------------------------------------------------------------------------------------------------
Final Balance             230   SATS
------------------------------------

In [9]:
print(balances["Customer Deposits Hive (Asset) - Sub: devser.v4vapp"])

Balance for Customer Deposits Hive (sub: devser.v4vapp) Asset
-----------------------------------------------------------------------------------------------------------------------------

Unit: HIVE
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-06-09 11:40:54  Treasury to Server transfer: 50.000 HIVE f...        50.00         0.00        50.00  96629458_6ce61
2025-06-10 07:39:21  4.445  HBD -->     18.292 HIVE droida fill...        18.29         0.00        68.29  96653393_ae62c
2025-06-10 08:28:12  Deposit: 16.000 HIVE from v4vapp-test to d...        16.00         0.00        84.29  96654370_79508
2025-06-10 08:28:50  Conv 15.509 HIVE to 3,434 sats devser.v4vapp          0.00        15.51        68.78      1f8eaa549a
2025-06-10 08:29:00  Withdrawal: 0.491 HIVE from devser.v4vapp ...         0.00         0.49        68.29  96654386_232d8
--------------------------------------------------------------

In [10]:
print(balances["Treasury Lightning (Asset) - Sub: umbrel"])


Balance for Treasury Lightning (sub: umbrel) Asset
-----------------------------------------------------------------------------------------------------------------------------

Unit: SATS
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-06-09 11:30:16  Funding for v2                                     100,000            0      100,000      Nl5k8e/nq4
2025-06-10 07:38:30  Conv 4.445 HBD to 4,069 sats devser.v4vapp           4,069            0      104,069      18642d3ac1
2025-06-10 07:38:30  Outgoing Lightning Payment 4.313 HBD 3,948...            0        3,948      100,120      18642d3ac1
2025-06-10 08:28:50  Conv 15.509 HIVE to 3,434 sats devser.v4vapp         3,434            0      103,554      1f8eaa549a
2025-06-10 08:28:50  Outgoing Lightning Payment 15.011 HIVE 3,3...            0        3,324      100,230      1f8eaa549a
-------------------------------------------------------------------------

In [11]:
print(balances["Treasury Lightning (Asset) - Sub: umbrel"])


Balance for Treasury Lightning (sub: umbrel) Asset
-----------------------------------------------------------------------------------------------------------------------------

Unit: SATS
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-06-09 11:30:16  Funding for v2                                     100,000            0      100,000      Nl5k8e/nq4
2025-06-10 07:38:30  Conv 4.445 HBD to 4,069 sats devser.v4vapp           4,069            0      104,069      18642d3ac1
2025-06-10 07:38:30  Outgoing Lightning Payment 4.313 HBD 3,948...            0        3,948      100,120      18642d3ac1
2025-06-10 08:28:50  Conv 15.509 HIVE to 3,434 sats devser.v4vapp         3,434            0      103,554      1f8eaa549a
2025-06-10 08:28:50  Outgoing Lightning Payment 15.011 HIVE 3,3...            0        3,324      100,230      1f8eaa549a
-------------------------------------------------------------------------